In [1]:
import plotly
import dash
import numpy as np
import pandas as pd

In [10]:
# Load the 4 datasets using the .read_csv function from pandas, sep = '\t' for tsv files, don't load the header
countries = pd.read_csv('Data/countries.tsv', sep='\t', header=0)
missions = pd.read_csv('Data/missions.tsv', sep='\t', header=0)
places = pd.read_csv('Data/places.tsv', sep='\t', header=0)
users = pd.read_csv('Data/users.tsv', sep='\t', header=0)

In [35]:
def get_other(co2):
    """
    Input: co2 value for trip
    Output: 'approximated' mode of travel
    Reoformat the mode when 'other' is used as a placeholder.
    We use the co2 to get the mode so we can save one lookup in the 'places' table
    """
    if co2 == 0.075:
        mode = "public"
    elif co2 == 0.20033:
        mode = "car"
    elif co2 == 0.01214:
        mode = "train"
    else:
        mode = "plane"
    return mode


In [31]:
def get_emission(placeid, co2km):
    """
    Input: Place ID, Co2 per km
    Output: total emission of the trip
    Here we retrieve from the places dataframe the distance and we
    return the total emissions as km*co2
    """
    for index, row in places.iterrows():
        if(places.at[index, '#place_id']) == placeid:
            return places.at[index, 'distance']*co2km
    print('Distance not found!')
    return 0


def get_region(userid):
    """
    Input: userID
    Output: Region of the user
    Here we retrieve from the users dataframe the region
    of the user #user_id
    """
    for index, row in users.iterrows():
        if(users.at[index, '#user_id']) == userid:
            return users.at[index, 'region']
    print('Region not found!')
    return 0

In [36]:
emissions, regions = [], []

for index, row in missions.iterrows():
    # Retrieve total emissions
    emissions.append(get_emission(missions.at[index, 'place_id'], missions.at[index, 'co2']))
    # Retrieve Houses for users
    regions.append(get_region(missions.at[index, 'user_id']))
    # Retrieve mode of travel when 'other' is selected -- design chioce explained in report, uncomment if displaying 'other'
    if(missions.at[index, 'mode'] == 'other'): 
        missions.at[index, 'mode'] = get_other(missions.at[index, 'co2']) 

missions['emissions'] = emissions
missions['regions'] = regions

In [37]:
missions

,#mission_id,user_id,place_id,date,duration,mode,co2,emissions,regions
0,0,299,55,2013-03-20,10,car,0.20033,18.83102,North
1,1,157,183,2013-03-08,16,train,0.01214,5.77864,Reach
2,2,491,191,2013-03-28,18,train,0.01214,5.83934,North
3,3,490,297,2013-03-17,75,plane,0.14678,108.76398,North
4,4,340,90,2013-03-19,14,train,0.01214,2.51298,North
...,...,...,...,...,...,...,...,...,...
5610,5610,341,183,2020-10-30,17,train,0.01214,5.77864,Dorne
5611,5611,137,387,2020-10-17,334,plane,0.14678,186.26382,Reach
5612,5612,450,1,2020-10-16,104,car,0.20033,0.20033,North
5613,5613,455,0,2020-11-08,84,public,0.07500,0.00000,North


In [69]:
# visit http://127.0.0.1:8050/ in your browser to run this visualization
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

df = px.data.gapminder()
all_continents = df.continent.unique()
print(all_continents)

modes = ['public', 'car', 'train', 'plane']

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div([
        html.H1('InfoViz assignment: Visualizing emission per region'),
        dcc.Checklist(
            id="checklist",
            options=[{"label": x, "value": x} 
                     for x in modes],
            value=modes[0:],
            labelStyle={'display': 'inline-block'}
        ),
        dcc.Graph(id="line-chart"),
        ])

@app.callback(
Output("line-chart", "figure"), 
[Input("checklist", "value")])
def update_line_chart(modes):
    mask = missions['mode'].isin(modes)
    fig = px.line(missions[mask], 
        x="date", y="emissions", color='regions')
    return fig


app.run_server(mode='inline')

['Asia' 'Europe' 'Africa' 'Americas' 'Oceania']


In [53]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [67]:
df['continent']

0         Asia
1         Asia
2         Asia
3         Asia
4         Asia
         ...  
1699    Africa
1700    Africa
1701    Africa
1702    Africa
1703    Africa
Name: continent, Length: 1704, dtype: object

In [68]:
missions['mode']

0          car
1        train
2        train
3        plane
4        train
         ...  
5610     train
5611     plane
5612       car
5613    public
5614     plane
Name: mode, Length: 5615, dtype: object